In [2]:
import pandas as pd
import sqlite3


#proof of concept - only has data from 9/7/19-9/21-19
conn = sqlite3.connect('mta_scrape1.db')
query = "SELECT * from entries"

df = pd.read_sql_query(query,conn)

df

,id,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,TIME,DESC,ENTRIES,EXITS
0,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 00:00:00,00:00:00,REGULAR,7198818,2438323
1,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 04:00:00,04:00:00,REGULAR,7198834,2438325
2,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 08:00:00,08:00:00,REGULAR,7198847,2438354
3,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 12:00:00,12:00:00,REGULAR,7198929,2438428
4,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 16:00:00,16:00:00,REGULAR,7199125,2438483
5,6,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 20:00:00,20:00:00,REGULAR,7199405,2438527
6,7,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-15 00:00:00,00:00:00,REGULAR,7199547,2438545
7,8,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-15 04:00:00,04:00:00,REGULAR,7199567,2438547
8,9,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-15 08:00:00,08:00:00,REGULAR,7199579,2438564
9,10,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-15 12:00:00,12:00:00,REGULAR,7199648,2438630


In [4]:
df['ENTRIES']

0         7198818
1         7198834
2         7198847
3         7198929
4         7199125
5         7199405
6         7199547
7         7199567
8         7199579
9         7199648
10        7199800
11        7200012
12        7200082
13        7200086
14        7200125
15        7200262
16        7200539
17        7201201
18        7201361
19        7201375
20        7201413
21        7201561
22        7201877
23        7202574
24        7202723
25        7202742
26        7202782
27        7202925
28        7203240
29        7203963
           ...   
615278       5554
615279       5554
615280       5554
615281       5554
615282       5554
615283       5554
615284       5554
615285       5554
615286       5554
615287       5554
615288       5554
615289       5554
615290       5554
615291       5554
615292       5554
615293       5554
615294       5554
615295       5554
615296       5554
615297       5554
615298       5554
615299       5554
615300       5554
615301       5554
615302    

In [ ]:
#create turnstile index column
df['TURNSTILE_INDEX'] = df['C/A'] + '-' +  df['UNIT'] + '-' + df['SCP'] + '-' + df['STATION']

df.head()

In [ ]:
#create summary of # of unique turnstiles/station
n_turnstiles = df.groupby('STATION')['TURNSTILE_INDEX'].nunique().sort_values(ascending=False)
                    
#top 10 stations by number of turnstiles    
n_turnstiles.head(10)